In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

# Hiperparametreler
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.0001
IMAGE_DIR = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
ATTR_PATH = "/kaggle/input/celeba-dataset/list_attr_celeba.csv"
PARTITION_PATH = "/kaggle/input/celeba-dataset/list_eval_partition.csv"

# Veri setini yükleme ve işleme
attr_df = pd.read_csv(ATTR_PATH)
partition_df = pd.read_csv(PARTITION_PATH)

# Smiling etiketini işaretleme
attr_df["smiling"] = attr_df["Smiling"].apply(lambda x: "smiling" if x == 1 else "not_smiling")

# Eğitim ve doğrulama veri çerçeveleri
partition_df = partition_df.rename(columns={"image_id": "image_id", "partition": "partition"})
train_df = partition_df[partition_df["partition"] == 0]
val_df = partition_df[partition_df["partition"] == 1]

train_df = train_df.merge(attr_df[["image_id", "smiling"]], on="image_id")
val_df = val_df.merge(attr_df[["image_id", "smiling"]], on="image_id")

# Veri dengesini kontrol et
train_class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df["smiling"]),
    y=train_df["smiling"]
)
class_weights = dict(enumerate(train_class_weights))

print(f"Class Weights: {class_weights}")

# Veri artırma
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Veri jeneratörleri
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=IMAGE_DIR,
    x_col="image_id",
    y_col="smiling",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory=IMAGE_DIR,
    x_col="image_id",
    y_col="smiling",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Modeli oluştur
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = False  # İlk olarak önceden eğitilmiş ağı dondur

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# Modeli derle
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "/kaggle/working/best_model.keras",
        save_best_only=True,
        monitor="val_accuracy",
        mode="max"
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-6
    )
]

# Modeli eğit
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights
)

# Temel modeli yeniden eğitme
base_model.trainable = True

# Yalnızca belirli katmanları yeniden eğit
for layer in base_model.layers[:100]:
    layer.trainable = False

# Modeli yeniden eğit
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE / 10),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_fine_tune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS // 2,
    callbacks=callbacks,
    class_weight=class_weights
)

# Modeli değerlendir
loss, accuracy = model.evaluate(val_generator)
print(f"Doğrulama Kaybı: {loss}")
print(f"Doğrulama Doğruluğu: {accuracy}")

# Modeli kaydet
model.save("/kaggle/working/final_model.keras")

# Eğitim geçmişini görselleştirme
plt.figure(figsize=(12, 6))

# Doğruluk
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Eğitim Doğruluğu")
plt.plot(history.history["val_accuracy"], label="Doğrulama Doğruluğu")
plt.title("Model Doğruluğu")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Kayıp
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Eğitim Kaybı")
plt.plot(history.history["val_loss"], label="Doğrulama Kaybı")
plt.title("Model Kaybı")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

Class Weights: {0: 0.9609753217617192, 1: 1.042328381147541}
Found 162770 validated image filenames belonging to 2 classes.
Found 19867 validated image filenames belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2544/2544 ━━━━━━━━━━━━━━━━━━━━ 5277s 2s/step - accuracy: 0.6989 - loss: 0.5704 - val_accuracy: 0.7900 - val_loss: 0.4512 - learning_rate: 1.0000e-04
Epoch 2/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4879s 2s/step - accuracy: 0.7712 - loss: 0.4776 - val_accuracy: 0.7960 - val_loss: 0.4394 - learning_rate: 1.0000e-04
Epoch 3/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4923s 2s/step - accuracy: 0.7810 - loss: 0.4626 - val_accuracy: 0.7998 - val_loss: 0.4290 - learning_rate: 1.0000e-04
Epoch 4/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4900s 2s/step - accuracy: 0.7859 - loss: 0.4531 - val_accuracy: 0.8044 - val_loss: 0.4248 - learning_rate: 1.0000e-04
Epoch 5/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4891s 2s/step - accuracy: 0.7915 - loss: 0.4467 - val_accuracy: 0.8067 - val_loss: 0.4201 - learning_rate: 1.0000e-04
Epoch 6/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4785s 2s/step - accuracy: 0.7901 - loss: 0.4465 - val_accuracy: 0.8084 - val_loss: 0.4187 - learning_rate: 1.0000e-04
Epoch 7/10
2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4836s 2s/